ADD INTRO

Here's the bit of code from the first part that we need for the examples in this part to work. 

*We can edit this part out of the final post, but I need it here in my Jupyter notebook.*

In [9]:
from django.apps import apps
from django.conf import settings
if not settings.configured:
    settings.configure(
     DEBUG=True,
     SECRET_KEY='thisisthesecretkey',
     MIDDLEWARE_CLASSES=(
     'django.middleware.common.CommonMiddleware',
     'django.middleware.csrf.CsrfViewMiddleware',
     'django.middleware.clickjacking.XFrameOptionsMiddleware',
     ),
    )
apps.populate([])

In [ ]:
from dataclasses import dataclass

@dataclass
class Thing:
    id: int
    b: str

    def __str__(self):
        return '<Thing(%d, "%s")>' % (self.id, self.b)

Changing an object
------------------

Let's see how we'd implement changing one of the fields on an existing Thing.

The way an API client might do this is to GET a URI path that points
to an existing Thing, change a value on its copy of the Thing, then
make a PUT request, using the same URI path, and putting the serialized
form of its edited Thing as the request body.

Here's how we might handle the PUT (ignoring for now the problem of
finding the existing thing to modify). Like `create`, we have to write our own `update` method.
Then we can pass both an instance and a data object to the serializer, and
saving will update the instance using the data by calling our
`update` method.

In [ ]:
from rest_framework import serializers

class ThingSerializer(serializers.Serializer):
    id = serializers.IntegerField(required=False)
    b = serializers.CharField()

    def validate_id(self, value):
        # Are we trying to create a new thing?
        if not self.instance and value:
            raise serializers.ValidationError('Cannot specify id when creating new thing')
        return value

    def create(self, validated_data):
        return Thing(**validated_data)

    def update(self, instance, validated_data):
        thing = instance
        thing.__dict__.update(validated_data)
        return thing

In [ ]:
existing_thing = Thing(id=27, b='three')
data = {'id': 13, 'b': 'three'}
serializer = ThingSerializer(instance=existing_thing, data=data)
serializer.is_valid(raise_exception=True)
updated_thing = serializer.save()
print(str(updated_thing))

DRF passes the validated data to our `update` method, the same as it
does for our `create` method, along with the original object.
Our `update` method must make changes to the original
object, then return it.

But again, this is too simple. If we pass an `id` value as part of
our data, it will happily change the `id` on our Thing. If this were
a Django model, that would not be what we'd want. So on update, it
turns out we also want to prohibit passing a value in our data for
`id`:

In [ ]:
from rest_framework import serializers

class ThingSerializer(serializers.Serializer):
    id = serializers.IntegerField(required=False)
    b = serializers.CharField()

    def validate_id(self, value):
        # Are we trying to create a new thing?
        if not self.instance and value:
            raise serializers.ValidationError('Cannot specify id when creating new thing')
        # Are we trying to update an existing thing?
        if self.instance and value:
            raise serializers.ValidationError('Cannot specify id when updating a thing')
        return value

    def create(self, validated_data):
        return Thing(**validated_data)

    def update(self, instance, validated_data):
        thing = instance
        thing.__dict__.update(validated_data)
        return thing

In [ ]:
existing_thing = Thing(id=27, b='three')
data = {'id': 13, 'b': 'three'}
print("Valid: %s" % serializer.is_valid())
print(serializer.errors)

But that's going too far. Our client would like to be able to GET the current
thing, change the value of `b`, then PUT the updated data. The way we have
written this, the client would be forced to first delete the `id` field from
the data. If the `id` field is unchanged, then it's harmless, so let's allow that.

In [ ]:
from rest_framework import serializers

class ThingSerializer(serializers.Serializer):
    id = serializers.IntegerField(required=False)
    b = serializers.CharField()

    def validate_id(self, value):
        # Are we trying to create a new thing?
        if not self.instance and value:
            raise serializers.ValidationError('Cannot specify id when creating new thing')
        # Are we trying to update an existing thing?
        if self.instance and value != self.instance.id:
            raise serializers.ValidationError('Cannot change id when updating a thing. Current id is %r, new id is %r' % (self.instance.id, value))
        return value

    def create(self, validated_data):
        return Thing(**validated_data)

    def update(self, instance, validated_data):
        thing = instance
        thing.__dict__.update(validated_data)
        return thing

In [ ]:
existing_thing = Thing(id=27, b='three')
data = {'id': 27, 'b': 'three'}
serializer = ThingSerializer(instance=existing_thing, data=data)
serializer.is_valid(raise_exception=True)

If this was a Django application and Thing was a model, then `update`
would also be expected to save the updated Thing before returning.

Notice that this time, we passed *both* an instance and some serialized data
to our serializer constructor. This tells it that we want to make changes to
the instance based on the serialized data.


We've changed the value of Thing's `id` field from 27 to 13.

## Validation

Validation is an area of DRF where I had to figure a lot out by trial and error.

Keep in mind that validation only applies to deserializing.

There are definite parallels between DRF validation and Django form validation,
but some important differences as well.

### DRF's field validation

The first thing that DRF does is validate the input data for each field defined
on the serializer. Any additional input data is simply ignored.

Some of this is really obvious, such as providing a string as
the value for an IntegerField is not valid.

If the data passes validation, then `.validated_data`, and the data passed
to `.create()` and `.update()`, will be a dictionary with a key for each field
defined on the serializer, whose value is the *serialized* data for that field.

*Note* this is a difference from Django forms. Part of Django form validation
is to convert the input data from the form into corresponding Python data
types, but DRF does not do this during validation.

## Nesting objects using keys

There are multiple ways we can represent nested objects when we serialize. Let's add another
class to our example and then briefly go over them.

In [ ]:
@dataclass
class Box:
    id: int
    thing: Thing

    def __str__(self):
       return '<Box(%d, "%s")>' % (self.id, self.thing)

The Box class has an identifier and a reference to a Thing.

Our Thing class has an `id` field, and if we assume for a moment that
these are Django models, we know that all we need is the `id` value and
we can fetch the entire Thing from the database. So we can serialize
a Box most simply by just using the `id` from our Thing:

In [ ]:
from rest_framework import serializers

class BoxSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    thing = serializers.IntegerField(source='thing.id')

The `source` argument on a field, as you might guess, in this case tells DRF to access the `id` attribute
on the `thing` attribute of the object being serialized.

Let's make a Box and serialize it.

In [ ]:
box = Box(2, Thing(id=5, b='drf'))
serializer = BoxSerializer(instance=box)
data = serializer.data
print(data)

This looks about as we'd expect.
DRF has made the value of 'thing' be the id from our thing.

As I hinted earlier, serializing is pretty straightforward. What
about deserializing? Let's add a `create` method.

In [ ]:
def get_existing_thing(id: int) -> Thing:
    """Dummy 'get_existing_thing' method."""
    return Thing(id, 'existing')

class BoxSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    thing = ThingSerializer()

    def create(self, validated_data):
        thing = get_existing_thing(id=validated_data['thing']['id'])
        return Box(id=validated_data['id'], thing=thing)

In [ ]:
thing = Thing(id=1, b='two')
ser = BoxSerializer(data={'id': 1, 'thing': 1})
if not ser.is_valid():
    print(ser.errors)
ser.is_valid(raise_exception=True)
box = ser.save()
print(box)
print(box.thing)

We're assuming some method `get_existing_thing(id=...)` does the heavy
lifting in finding an existing Thing for us. In Django, it would probably
just be `Thing.objects.get(id=id)`.

Here's a place where DRF's behavior confuses me.

When we validate the data in which we've just represented the Thing
as an `IntegerField` with `source='thing.id'`, validation gives us back
not the same integer as I was expecting, but a dictionary with
`{'id': <value of integer field>}`.  That's why we had to use
`validated_data['thing']['id']` to get the value of `id` and not
just `validated_data['id']`.

Continuing with serializing the `thing` field of our Box as just
the `id` value of our `Thing`, what if we want to update our Box?
Keeping in mind that for the `thing` field, all we can really update this way
is which Thing our box is pointing at, we need to add
an ``update`` method to our serializer again::

In [ ]:
class BoxSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    thing = serializers.IntegerField(source='thing.id')

    def create(self, validated_data):
        thing = get_existing_thing(id=validated_data['thing']['id'])
        return Box(id=validated_data['id'], thing=thing)

    def update(self, instance, validated_data):
        instance.id = validated_data['id']
        instance.thing = get_existing_thing(id=validated_data['thing']['id'])
        return instance

Let's try it out:

In [ ]:
thing1 = Thing(1, "thing1")
thing2 = Thing(2, "thing2")
box = Box(3, thing1)
print(str(box))

data = {'id': 3, 'thing': 2}
serializer = BoxSerializer(instance=box, data=data)
serializer.is_valid(raise_exception=True)
box = serializer.save()
print(str(box))

We can see that the box was changed to point at a different Thing.

## Nested serialized objects

Representing nested objects using a key works well. After all, relational databases
have worked this way for decades. It might mean a few more queries, since we might have
to make another call to get the object represented by a key or update it, but generally our
application code has had a short network path to our database server, so the queries
can be pretty quick.

But in a modern app, our API's client might be a user's
browser, and that might be a long way from our server, compared to the network distance
between our web server and our database. So requiring many small API calls to get anything
done, where each call requires the latency of a complete round trip between the browser
and our web server, could have a negative effect on our app's performance.

This is one reason why we might consider it worthwhile to add the complexity
to our API of nesting entire serialized objects.

To make things more concrete, I'm going to start a new series of examples.

In [ ]:
from dataclasses import dataclass
from decimal import Decimal

from rest_framework import serializers

@dataclass
class Person:
    id: int
    name: str
    email: str

@dataclass
class Account:
    id: int
    owner: Person
    balance: Decimal

class PersonSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    name = serializers.CharField()
    email = serializers.EmailField()
    
class AccountSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    balance = serializers.DecimalField(max_digits=10, decimal_places=2)
    owner = PersonSerializer()

Here we have very simple Person and Account classes, where the Account has a Person as owner.

Anywhere you might use a serializer field in DRF, you can instead use a serializer.
I'd have prefered to keep the two types more distinct (but nobody asked me).

So if we want to nest a serialized person in our serialized account, we just
tell DRF that the owner field on the account should be serialized using
`PersonSerializer`.

In [ ]:
person = Person(id=1, name='Fred', email='fred@example.com')
account = Account(id=1, balance=Decimal(50), owner=person)
print(AccountSerializer(instance=account).data)

As we've seen before, serializing is generally pretty straightforward. But with
entire nested objects, deserializing and creating a new object will have some
new complications.  Let's update our serializers with basic `create` methods.

In [ ]:
class PersonSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    name = serializers.CharField()
    email = serializers.EmailField()
    
    def create(self, validated_data):
        return Person(**validated_data)
    
class AccountSerializer(serializers.Serializer):
    balance = serializers.DecimalField(max_digits=10, decimal_places=2)
    owner = PersonSerializer()
    
    def create(self, validated_data):
        owner_serializer = PersonSerializer(data=validated_data.pop('owner'))
        owner_serializer.is_valid(raise_exception=True)
        validated_data['owner'] = owner_serializer.save()
        return Account(**validated_data)

We have some new code in `AccountSerializer.create` that first creates a person from
the 'person' part of the validated_data, then uses that in creating the account. It's
a little more code, but still fairly straightforward.

But let's think about this a bit. This assumes that every time we create a new Account,
we also want to create a new Person at the same time. That's probably not a valid
assumption.

Worse, 

NOW:
- how does DRF validate the new field itself?
- what gets passed to 'validate'
- what gets passed to 'create'/'update'?
